1. Carga y Exploración de Datos:

Cargar el dataset y revisar la estructura básica.
Descripción de las variables y su distribución.
Detección y tratamiento de valores nulos.
Identificación y tratamiento de outliers.
Análisis de correlación entre variables.

In [1]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv("../Data/Automobile_data.csv")  

In [ ]:
df.head(12).T 

In [ ]:
df.info() 
df.describe() 

Hay algunos datos que figuran como object en vez de como corresponden. Esto es porque no aparecen como NaN y aparecen como ?
Se prosede a arregrar remplazando por nan y luego para identificar valores nulos

In [ ]:
df.replace('?', np.nan, inplace = True) 
print(df.isnull().sum()) 

In [ ]:

#Luego de verificar el dataset se considera a estas columnas como numericas que estan como object
df['normalized-losses'] = pd.to_numeric(df['normalized-losses'], errors = 'coerce')
df['bore'] = pd.to_numeric(df['bore'], errors='coerce')
df['stroke'] = pd.to_numeric(df['stroke'], errors='coerce')

#normalized-losses lo completaré con el promedio
df['normalized-losses'] = df['normalized-losses'].fillna(df['normalized-losses'].mean())
#numero de puertas con moda
df['num-of-doors'] = df['num-of-doors'].fillna(df['num-of-doors'].mode()[0])
#bore y stroke también hare con el promedio

df['bore'] = df['bore'].fillna(df['bore'].mean())
df['stroke'] = df['stroke'].fillna(df['stroke'].mean())
#Horsepower, peak rpm y price considero que es muy importante entonces no lo promedio, mejor lo elimino
df.dropna(subset = ['horsepower', 'peak-rpm', 'price'], inplace = True)
#Ahora verifico
print(df.isnull().sum())


In [ ]:
print(df.dtypes)

In [ ]:
#Modifiquemos los tipos de datos de las columnas que sean necesarias
df['normalized-losses'] = df['normalized-losses'].astype(int)
df['num-of-doors'] = df['num-of-doors'].astype('category')
df['horsepower'] = df['horsepower'].astype(int)
df['peak-rpm'] = df['peak-rpm'].astype(int)
print(df.dtypes)

In [ ]:
#Ahora trataremos los outliers
sns.boxplot(data = df[['horsepower', 'city-mpg', 'highway-mpg']])
plt.show()

In [ ]:
#Considero que estos valores son posibles así que no los elimino
#Ahora haremos un análisis de correlación entre las variables
df_numeric = df.select_dtypes(include = ['int64', 'float64'])
corr_matrix = df_numeric.corr(method = 'pearson') #si no pongo el método el usará pearson por defecto
plt.figure(figsize = (12, 10))
sns.heatmap(corr_matrix, annot = True, cmap = 'coolwarm')
plt.show()

 2. Preprocesamiento:

Selección de características importantes.
Transformación de variables categóricas.
División del conjunto de datos en entrenamiento y prueba.
Escalado de características.

In [ ]:
results = {} #Esto es para almacenar los resultados
#Ahora seleccionaremos las variables que vamos a usar
X = df.drop(['price'], axis = 1) #Con esto selecciono todo menos price
y = df['price'].astype(float)

#Dividimos el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

#Debemos dividir las columnas categóricas y numéricas
categorical_features = X.select_dtypes(include = ['object']).columns
numerical_features = X.select_dtypes(include=['number']).columns

#Creamos el preprocesador usando ColumnTransformer
preprocessor = ColumnTransformer(
    transformers = [
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown = 'ignore'), categorical_features)
    ]
)


3. Regresión Lineal:

Entrenamiento del modelo.
Evaluación del rendimiento (MSE y R²).
 

4. K-Nearest Neighbors (KNN):

Entrenamiento del modelo.
Evaluación del rendimiento (MSE y R²).

5. Árbol de Decisión:

Entrenamiento del modelo.
Evaluación del rendimiento (MSE y R²).


In [ ]:
#Creamos el pipeline para la regresión lineal
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

#Ajustamos el modelo a los datos de entrenamiento
pipeline.fit(X_train, y_train)

#Realizamos las predicciones
y_pred = pipeline.predict(X_test)
#Evaluamos el rendimiento (MSE y R2)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
results['Regresión lineal'] = {'MSE': mse, 'R2': r2}
print(f'Regresión lineal - MSE: {mse:.2f}, R2: {r2:.2f}')

#Ahora vamos con el K-Nearest Neighbors
#Creamos el pipeline para el KNN
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', KNeighborsRegressor(n_neighbors = 5))
])

#Entrenamos el modelo
pipeline.fit(X_train, y_train)

#Realizamos las predicciones
y_pred = pipeline.predict(X_test)

#Evaluamos el rendimiento (MSE y R2)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
results['KNN'] = {'MSE': mse, 'R2': r2}
print(f'KNN - MSE: {mse:.2f}, R2: {r2:.2f}')

#Ahora vamos el el árbol de decisión
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', DecisionTreeRegressor(random_state = 0))
])

#Entrenamos el modelo
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

#Evaluamos el rendimiento (MSE y R2)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
results['Arbol de decisión'] = {'MSE': mse, 'R2': r2}
print(f'Arbol de decisión: - MSE: {mse:.2f}, R2: {r2:.2f}')

6. Comparación de Modelos:

Comparar los resultados de los tres modelos en términos de MSE y R².
Discusión sobre las diferencias en el rendimiento de los modelos.


In [ ]:
#Ahora comparamos los resultados de los tres modelos
#podemos hacer un dataframe con los resultados
results_df = pd.DataFrame(results).T
print(results_df)

7. Interpretación de Resultados:

Analizar cuál de los modelos es más adecuado para el problema de predicción de precios de autos.
Discutir posibles mejoras y próximos pasos, como el ajuste de hiperparámetros o el uso de técnicas avanzadas.

# Regresión Lineal
MSE extremadamente alto: Esto indica que el modelo no está prediciendo correctamente. Es probable que la relación entre las variables no sea lineal, lo que causa un desempeño tan deficiente.
R² negativo y muy bajo: Un R² negativo significa que el modelo es peor que simplemente predecir el promedio de y. Esto confirma que la Regresión Lineal no es adecuada para este conjunto de datos.
Conclusión: Este modelo no es útil para este problema, ya que no captura adecuadamente la relación entre las variables.

# KNN (K-Nearest Neighbors)
MSE moderadamente bajo: Aunque no es el modelo más preciso, el error promedio es razonablemente bajo en comparación con la Regresión Lineal.
R² positivo (0.6519): Indica que el modelo explica el 65.19% de la variabilidad en los datos. Esto sugiere un desempeño moderado, pero no óptimo.
Conclusión: El modelo KNN funciona mejor que la Regresión Lineal, pero podría no ser el ideal para este problema. Ajustar el número de vecinos podría mejorar el desempeño.

# Árbol de Decisión
MSE más bajo entre los tres modelos: este modelo tiene el menor error promedio, lo que indica que predice más precisamente.
R² más alto: El modelo explica el 88.21% de la variabilidad en los datos, lo que sugiere que captura bien las relaciones no lineales en el conjunto.
Conclusión: El Árbol de Decisión es el modelo más adecuado entre los tres, ya que tiene el menor error y explica la mayor parte de la variabilidad en los datos.

Conclusión general:
El Árbol de Decisión es el modelo más efectivo para este problema, ya que:

Tiene el menor MSE, indicando predicciones más precisas.
Posee el mayor R² (0.8821), lo que sugiere que es capaz de capturar mejor las relaciones complejas en los datos.

Sin embargo, sería prudente considerar lo siguiente:

Validación cruzada: Para confirmar que el modelo generaliza bien y evitar el riesgo de sobreajuste.
Hiperparámetros: Explorar ajustes en la profundidad máxima del árbol (max_depth) o en los criterios de división para afinar el modelo.
Otros modelos: Puedes probar técnicas avanzadas como Random Forest o Gradient Boosting para mejorar aún más el rendimiento.